# Editing arguments and attacks

## Initialization

In [22]:
from incraf import IncrAFSolver

from clingo.control import Control
from clingo.solving import Model

from clingox.program import Program, ProgramObserver

In [23]:
def dump_ground_program(solver):
    print(solver.prg)

def on_model(m: Model):
    print(f"Model {m.number}:", m)

## Loading of the AF

In [24]:
filename = "asp/af1.lp"
semantic = "naive"

In [25]:
ctl1 = Control(arguments=["--models=0"])
prg1 = Program()
ctl1.register_observer(ProgramObserver(prg1))

ctl1.load(filename)
ctl1.load(f"dung/{semantic}.dl")
ctl1.load("asp/filter.lp")

ctl1.ground()

af = IncrAFSolver(semantic, filename)
# dump_ground_program(af)

We call `solve_enum` for the initial AF, which produces all naive extensions in this case.

In [26]:
print("=== Static solver ===")
ctl1.solve(on_model=on_model)

print("\n=== Incremental solver ===")
af.solve_enum(verbose=True);

=== Static solver ===
Model 1: in(a) in(c)
Model 2: in(b)

=== Incremental solver ===
DEBUG Model 1: in(b)
DEBUG Model 2: in(a) in(c)


## Editing operations
Next, we try to transfrom the initial AF (as in `asp/af1.lp`) into the 
AF as in `asp/af3.lp` (a cycle of length 4).

In [27]:
af.add_argument("d")
af.del_attack("b", "a")
af.del_attack("c", "b")
af.add_attack("c", "d")
af.add_attack("d", "a")
af.add_attack("b", "c")

RuntimeError: redefinition of atom <'okOut(c)',12>

In [ ]:
# dump_ground_program(af)

Now we load `asp/af3.lp` statically to compare the output.

In [ ]:
ctl2 = Control(arguments=["--models=0"])
prg2 = Program()
ctl2.register_observer(ProgramObserver(prg2))

ctl2.load("asp/af3.lp")
ctl2.load(f"dung/{semantic}.dl")
ctl2.load("asp/filter.lp")

ctl2.ground()

In [ ]:
print("=== Static solver ===")
ctl2.solve(on_model=on_model)

print("\n=== Incremental solver ===")
af.solve_enum(verbose=True);

=== Static solver ===
Model 1: in(b) in(d)
Model 2: in(a) in(c)

=== Incremental solver ===
DEBUG Model 1: in(a) in(c) in(d)
DEBUG Model 2: in(b) in(c) in(d)
